In [1]:
import numpy
import os
import pandas
import cv2
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, Input,, Dropout, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16

In [2]:
IMG_SIZE = 256
sigmaX = 30

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[numpy.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][numpy.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][numpy.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][numpy.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][numpy.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = numpy.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [3]:
aptos_df = pandas.read_csv("../input/aptos2019-blindness-detection/train.csv")
dr_df = pandas.read_csv("../input/diabetic-retinopathy-resized/trainLabels.csv")

In [ ]:
aptos_dir = "../input/aptos2019-blindness-detection/train_images/"
dr_dir = "../input/diabetic-retinopathy-resized/resized_train/resized_train/"

images = []
labels = []
count0 = 0

# for idx in tqdm(range(len(dr_df))):
#     if dr_df.iloc[idx]['level'] == 0:
#         if count0 < 6038:
#             images.append(load_ben_color(dr_dir+dr_df.iloc[idx]['image']+'.jpeg'))
#             labels.append(dr_df.iloc[idx]['level'])
#             count0 += 1
#     else:
#         images.append(load_ben_color(dr_dir+dr_df.iloc[idx]['image']+'.jpeg'))
#         labels.append(dr_df.iloc[idx]['level'])

for idx in tqdm(range(len(aptos_df))):
    images.append(load_ben_color(aptos_dir+aptos_df.iloc[idx]['id_code']+'.png'))
    labels.append(aptos_df.iloc[idx]['diagnosis'])
    

 64%|██████▎   | 2327/3662 [07:50<03:25,  6.48it/s]

In [ ]:
images = numpy.array(images)
labels = numpy.array(labels)
cat_labels = to_categorical(labels)

In [ ]:
model = VGG16(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = GlobalAveragePooling2D()(model.output)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dense(32, activation='relu')(x)
op = Dense(5, activation='softmax')(x)

model = Model(inputs=model.inputs, outputs=op)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.fit(images, cat_labels, batch_size=32, epochs=8, shuffle=True, validation_split=0.1)

In [ ]:
model = ResNet50(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = GlobalAveragePooling2D()(model.output)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dense(32, activation='relu')(x)
op = Dense(5, activation='softmax')(x)

model = Model(inputs=model.inputs, outputs=op)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.fit(images, cat_labels, batch_size=32, epochs=8, shuffle=True, validation_split=0.1)

In [ ]:
model = Xception(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = GlobalAveragePooling2D()(model.output)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dense(32, activation='relu')(x)
op = Dense(5, activation='softmax')(x)

model = Model(inputs=model.inputs, outputs=op)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.fit(images, cat_labels, batch_size=32, epochs=8, shuffle=True, validation_split=0.1)